In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

from matplotlib import animation, rc
from IPython.display import HTML
rc('animation', html='jshtml')
import seaborn as sns

#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=2.2)

## Problema (Mola - plano inclinado)

<font size=3px>Um bloco de massa $m=0,9$ kg parte com velocidade inicial $v_0$ (paralela ao plano)
    e desliza sobre um plano inclidano sem atrito que faz um ângulo de $\theta = 30^{\circ}$ com a horizontal,
    como mostrado na figura. Após deslizar uma distância $d= 70$ cm o bloco encontra uma mola
    não deformada e desliza mais 30 cm até que sua velocidade instantânea seja zero. A constante
    elástica da mola é $k = 190$ N/m. Adote $g = 10$ m/s$^2$.<br>
    a) Qual a velocidade do bloco ao encontrar a mola? Escreva a sua resposta em termos de $v_0$.<br>
    b) Qual a velocidade inicial do bloco ($v_0$)?
</font>
<img src="molaInclinado.png" alt="Drawing" style="width: 250px;"/>

In [ ]:
theta = 30.*np.pi/180.
m = 0.9
g = 10.0
d = 0.7
k = 190.
dl = 0.3

def F(t,xv):
    x,v = xv
    if x < d:
        return m*g*np.sin(theta)
    else:
        return m*g*np.sin(theta)-k*(x-d)

def a(t,xv):
    return F(t,xv)/m

def RHS(t,xv):
    return [xv[1],a(t,xv)]

v0 = np.sqrt(k*(dl**2)/m - 2.*g*(d+dl)*np.sin(theta))
x0 = 0.
xv0 = [x0,v0]
t0, tf = 0.,1.5
sol = scipy.integrate.solve_ivp(RHS,(t0,tf),xv0,rtol=1e-3,atol=1e-5,max_step=0.01)
xmax,xmin = max(sol.y[0,:]),min(sol.y[0,:])
xF = scipy.interpolate.interp1d(sol.t,sol.y[0,:])
vF = scipy.interpolate.interp1d(sol.t,sol.y[1,:])

def KF(t):
    return (m*vF(t)**2)/2.

def UkF(t):   
    return k*(np.heaviside(xF(t)-d,1.)*(xF(t)-d)**2)/2.

def UgF(t):
    return m*g*(xmax-xF(t))*np.sin(theta)

def UF(t):
    return UkF(t)+UgF(t)

In [ ]:
tmin, tmax = t0,tf
dt = 0.05
tpts = np.linspace(tmin,tmax,100)

fig = plt.figure(figsize=(22,7))
ax1 = plt.subplot(131)
pt, = ax1.plot([xF(tmin),0.],'o',markersize=25)
line, = ax1.plot([xmax,max(xF(tmin),d)],[0.,0.],'--')
xlabel = ax1.set_xlabel('x (m)')
ax1.set_xlim(xmin,xmax)
ax1.set_ylim(-0.1,0.1)

ax2 = plt.subplot(132)
Kl, = ax2.plot(tpts,KF(tpts),'--',linewidth=4.0)
Ukl, = ax2.plot(tpts,UkF(tpts),'--',linewidth=4.0)
Ugl, = ax2.plot(tpts,UgF(tpts),'--',linewidth=4.0)
El, = ax2.plot(tpts,KF(tpts)+UF(tpts),linewidth=4.0)
xlabel = ax2.set_xlabel('t (s)')
ax2.legend(['K','Uk','Ug','K+U'],loc='upper right',framealpha=1.0,fontsize=25.0)


ax3 = plt.subplot(133)
Ufunc, = ax3.plot(xF(tpts),UF(tpts),'-',linewidth=4.0)
ptB, = ax3.plot([xF(tmin)],[UF(tmin)],'o',markersize=20)
ax3.plot([xmin,xmax],[KF(tmin)+UF(tmin),KF(tmin)+UF(tmin)],'-',linewidth=3)
xlabel = ax3.set_xlabel('x (m)')
ylabel = ax3.set_ylabel('U(x) (J)')



def init():
    tpts = [tmin]
    pt.set_data([xF(tmin)],[0.])
    line.set_data([xmax,max(xF(tmin),d)],[0.,0.])
    line._linewidth = 3.0
    Kl.set_data(tpts,KF(tpts))
    Ukl.set_data(tpts,UkF(tpts))    
    Ugl.set_data(tpts,UgF(tpts))
    ptB.set_data([xF(tmin)],[UF(tmin)])
    
    return (pt,line,Kl,Ukl,Ugl,ptB)

def animate(i):
    
    tA = tmin+i*dt
    tpts = np.linspace(tmin,tA,100)
    xpt = [xF(tA)]
    ypt = [0.]
    pt.set_data(xpt,ypt)
    line.set_data([xmax,max(xF(tA),d)],[0.,0.])
    if xF(tA) > d:
        line._linewidth = 4.0/(0.1+abs(xF(tA)-d)/2.)
    else:
        line._linewidth = 4.0/0.1
    Kl.set_data(tpts,KF(tpts))
    Ugl.set_data(tpts,UgF(tpts))    
    Ukl.set_data(tpts,UkF(tpts))
    ptB.set_data([xF(tA)],[UF(tA)])
    return (pt,line,Kl,Ukl,Ugl,ptB)


anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=int((tmax-tmin)/dt), interval=50, 
                               blit=True)
plt.close()
anim
# plt.show()

#### Respostas:

In [ ]:
vF = np.sqrt(v0**2 + 2*g*d*np.sin(theta))
v0 = np.sqrt(k*(dl**2)/m - 2.*g*(d+dl)*np.sin(theta))
print('a) vF = %2.3f m/s' %vF)
print('b) v0 = %2.3f m/s' %v0)